In [19]:
import pandas as pd

okc_basic = pd.read_csv('2024 OKC Basic.csv')
okc_advanced = pd.read_csv('2024 OKC Advanced.csv')
nba_salaries = pd.read_csv('nba_salaries.csv')

nba_salaries.columns = nba_salaries.iloc[0]
nba_salaries = nba_salaries[1:]
okc_rookies = nba_salaries[(nba_salaries['Tm'] == 'OKC') & 
                           (nba_salaries['2024-25'].notna()) & 
                           (nba_salaries['2025-26'].isna())]

okc_rookies['2024-25'] = pd.to_numeric(okc_rookies['2024-25'].str.replace('[\$,]', '', regex=True), errors='coerce')

rookies_names = okc_rookies['Player'].tolist()

okc_basic_rookies = okc_basic[okc_basic['Player'].isin(rookies_names)]
okc_advanced_rookies = okc_advanced[okc_advanced['Player'].isin(rookies_names)]
combined_rookies_stats = pd.concat([okc_basic_rookies.set_index('Player'), okc_advanced_rookies.set_index('Player')], axis=1)

combined_rookies_stats.reset_index(inplace=True)
output_file_path = 'OKC_Rookies.csv'
combined_rookies_stats.to_csv(output_file_path,index=False)


/var/folders/60/4pm0cms92nxg2ttn34jwc9800000gn/T/ipykernel_30053/1305161885.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  okc_rookies['2024-25'] = pd.to_numeric(okc_rookies['2024-25'].str.replace('[\$,]', '', regex=True), errors='coerce')


In [20]:
import pandas as pd

draft_2016_data = pd.read_csv('NBA_Draft_2016.csv')
draft_2017_data = pd.read_csv('NBA_Draft_2017.csv')
draft_2018_data = pd.read_csv('NBA_Draft_2018.csv')
salaries_data = pd.read_csv('nba_salaries.csv')

def clean_dataset(data):
    data.columns = data.iloc[0]  
    return data.drop(0).reset_index(drop=True)

draft_2016_data = clean_dataset(draft_2016_data)
draft_2017_data = clean_dataset(draft_2017_data)
draft_2018_data = clean_dataset(draft_2018_data)
salaries_data = clean_dataset(salaries_data)
salaries_data_relevant = salaries_data[['Player', '2023-24']]

merged_2016 = pd.merge(draft_2016_data, salaries_data_relevant, on='Player', how='inner')
merged_2017 = pd.merge(draft_2017_data, salaries_data_relevant, on='Player', how='inner')
merged_2018 = pd.merge(draft_2018_data, salaries_data_relevant, on='Player', how='inner')
concatenated_data = pd.concat([merged_2016, merged_2017, merged_2018], ignore_index=True)
output_file_path = 'Merged_NBA_Drafts_and_Salaries_2016_2018.csv'
concatenated_data.to_csv(output_file_path,index=False)

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import numpy as np

okc_rookies_data = pd.read_csv('OKC_Rookies.csv')
training_data = pd.read_csv('Merged_NBA_Drafts_and_Salaries_2016_2018.csv')
features = ['G', 'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'BPM', 'VORP']
target = '2023-24'

training_data[target] = training_data[target].replace('[\$,]', '', regex=True).astype(float)
imputer = SimpleImputer(strategy='median')
training_data_imputed = imputer.fit_transform(training_data[features + [target]])
okc_rookies_data_imputed = imputer.fit_transform(okc_rookies_data[features])

X = training_data_imputed[:, :-1]  
y = training_data_imputed[:, -1]  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
okc_scaled = scaler.transform(okc_rookies_data_imputed)

model = LinearRegression()
model.fit(X_train_scaled, y_train)
predicted_salaries = model.predict(okc_scaled)
print(predicted_salaries)


[ 8545056.01139248  9566838.36250267 10406023.72065276 11845816.24960353]
